# Assignment 11
## Due July 11 at 12:00

Please note: 

- Read the instructions in the exercise PDF and in this notebook carefully.
- Add your solutions *only* at `YOUR CODE HERE`/`YOUR ANSWER HERE` and remove the corresponding `raise NotImplementedError()`.
- Do not chance the provided code and text, if not stated.
- Do not *add* or *delete* cells.
- Do not `import` additional functionality. 
- Before submitting: Please make sure, that your notebook can be executed from top to bottom `Menu -> Kernel -> Restart & Run all`. 

## Exercise 1

## a)

Let $F$ be a finite function class. Let $X$ be a finite set and $n$ be the largest number of points s.t. $X_1 \dots X_n \in X$ is shattered by $F$ ($VC(F)=n$).

$\Rightarrow \mathcal{N}(\mathcal{F}, n) = 2^{n}$

$\Rightarrow |F| \geq \mathcal{N}(\mathcal{F}, n) = 2^{n}$

$\Rightarrow VC(F) \leq \log_2(|F|)$

In [1]:
import numpy as np
from numpy.random import normal, binomial, beta, standard_t
import matplotlib.pyplot as plt

np.random.seed(42)
%matplotlib inline

## Exercise 4

In [2]:
nb_samples = 10 ** 5
ks = [1, np.sqrt(2), 1.5, 2, 2*np.sqrt(2), 3, 4, 5]

### a) Sample from the binomial distribution

In [3]:
n = 50
p = 0.5

# sample xs
# YOUR CODE HERE

### Chebyshev's inequality.

In [1]:
# YOUR CODE HERE

### Plot the empirical distribution and indicate the Chebyshev bounds

In [6]:
def empirical_exceedance(xs, mu, bound):
    # calculates the minimal deviation from mu that fulfills the bound.
    # YOUR CODE HERE

In [2]:
# YOUR CODE HERE

### b) Sample from the beta distribution

In [8]:
nb_samples = 10 ** 2
m = 1000

In [3]:
a = 0.5    # alpha parameter for beta distribution
b = 0.5    # beta parameter for beta distribution

# sample xs
# YOUR CODE HERE

### Hoeffding's inequality

In [4]:
epsilons = np.linspace(0.01, 0.1, 10)

# YOUR CODE HERE

NameError: name 'np' is not defined

### Plot the empirical distribution and indicate the Hoeffding bounds

In [5]:
# plot histogram, Hoeffding's bounds and empirical bounds for eps in epsilons

# YOUR CODE HERE

### c) Student-t distribution with 10 degrees of freedom

In [13]:
nb_samples = 10 ** 2
m = 1000

In [6]:
epsilons = np.linspace(0.05, 0.1, 11)
xs = standard_t(10, size=(m, nb_samples))

# YOUR CODE HERE

NameError: name 'np' is not defined

### Why does the result occur?